In [ ]:
!pip install python-google-places

  Preparing metadata (setup.py) ... done
  Created wheel for python-google-places: filename=python_google_places-1.4.2-py3-none-any.whl size=13574 sha256=c2d85f8249c96364c52c722c92913d9e39a787a4274f9e0827f7b56f4bead5e1
  Stored in directory: /root/.cache/pip/wheels/6f/13/5a/c6b93902b894601cce5ba2194353ccb77d14157b78a8cfb748
Successfully built python-google-places


In [ ]:
import concurrent.futures
import time
import pandas as pd
from googleplaces import GooglePlaces

In [ ]:
API_key = "-----------------------------------"
google_places = GooglePlaces(API_key)

In [ ]:
# Load and process the CSV
df = pd.read_csv(r'/content/Coverage.csv')
area = df[['District', 'Upazila_or_Thana']].drop_duplicates().reset_index(drop=True)
locations_list = [', '.join(map(str, item)) for item in area.values.tolist()]

In [ ]:
len(locations_list)

560

In [ ]:
def fetch_restaurant_data(location):
    scraped_restaurants = []
    try:
        location_result = google_places.nearby_search(location=location, keyword='Restaurant', radius=500)
        if location_result:
            for place in location_result.places:
                place.get_details()
                scraped_restaurants.append([
                    place.details.get('place_id'),
                    place.name,
                    place.geo_location.get('lat'),
                    place.geo_location.get('lng'),
                    place.rating,
                    place.details.get('user_ratings_total'),
                    place.details.get('price_level'),
                    place.formatted_address
                ])
            time.sleep(2)
            while location_result.has_next_page_token:
                location_result = google_places.nearby_search(location=location, keyword='Restaurant',
                                                              radius=500, pagetoken=location_result.next_page_token)
                for place in location_result.places:
                    place.get_details()
                    scraped_restaurants.append([
                        place.details.get('place_id'),
                        place.name,
                        place.geo_location.get('lat'),
                        place.geo_location.get('lng'),
                        place.rating,
                        place.details.get('user_ratings_total'),
                        place.details.get('price_level'),
                        place.formatted_address
                    ])
            time.sleep(1)
    except Exception as e:
        print(f"Error fetching data for {location}: {e}")
    return scraped_restaurants

In [ ]:
def save_to_csv(data, filename="restaurants.csv"):
    df = pd.DataFrame(data, columns=["Place ID", "Name", "Latitude", "Longitude", "Rating", "Number of Reviews", "Price Level", "Address"])
    df.to_csv(filename, mode='a', header=not pd.io.common.file_exists(filename), index=False)


In [ ]:
# Use ThreadPoolExecutor to fetch data concurrently
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    futures = {executor.submit(fetch_restaurant_data, location): location for location in locations_list}
    for future in concurrent.futures.as_completed(futures):
        location = futures[future]
        try:
            data = future.result()
            save_to_csv(data)
            print(f"Data saved for {location}")
        except Exception as e:
            print(f"Error processing {location}: {e}")

Data saved for Pirojpur, Kawkhali
Data saved for Pirojpur, Nazirpur
Data saved for Pirojpur, Nesarabad
Data saved for Pirojpur, Pirojpur Sadar
Data saved for Pirojpur, Mathbaria
Data saved for Pirojpur, Zianagar
Data saved for Brahmanbaria, Ashuganj
Data saved for Brahmanbaria, Akhaura
Data saved for Brahmanbaria, Brahmanbaria Sadar
Data saved for Brahmanbaria, Bancharampur
Data saved for Brahmanbaria, Bijoynagar
Data saved for Brahmanbaria, Kasba
Data saved for Brahmanbaria, Nabinagar
Data saved for Brahmanbaria, Nasirnagar
Data saved for Brahmanbaria, Sarail
Data saved for Bandarban, Alikadam
Data saved for Bandarban, Lama
Error fetching data for Bandarban, Bandarban Sadar: Request to URL https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=22.1960332%2C92.21760630000001&radius=500&keyword=Restaurant&pagetoken=AelY_Ct3Wja5c9s7XkL7OpTGyKxUKYqPqFAdtdnLkmO7_iDXE40yfNnPd8cFxdt8XP0lMiTzm8sv6FS866aEzUtMnCMr2JJHYwv2fIodCXX7JmyC-Y6mbGmfkzio8sj8R_ZZ5t-G7klh59UmJLrHtMsO1qvWDP_

In [ ]:
restaurant_df = pd.read_csv(r'/content/restaurants.csv');

In [ ]:
restaurant_df.head()

,Place ID,Name,Latitude,Longitude,Rating,Number of Reviews,Price Level,Address
0,ChIJ2fWyft1VVTcRq-QNa_pzsx4,Al kayed mini chinese and resturent,22.627131,90.063290,4.2,10.0,NaN,"Baily Bridge, Kawkhali, Bangladesh"
1,ChIJPX6x-n1VVTcR0d1ozd5Rigs,মৃত্তিকা ফুড পার্ক,22.625809,90.070005,3.0,8.0,NaN,"eko park, Kawkhali, Bangladesh"
2,ChIJQ7sIqij9_zkRSkvhhE8AEZk,Friend Coffee Zone,22.786716,89.981345,0.0,NaN,NaN,"QXPJ+MGQ, Malikhali Union, Bangladesh"
3,ChIJOXSH65P9_zkRwa8cmONLl0U,অমৃত মৃষ্টান্ন ভান্ডার,22.786716,89.981345,0.0,NaN,NaN,"QXPJ+MGQ, Malikhali Union, Bangladesh"
4,ChIJ84dwtWb9_zkRQvgbrrra8ms,সুমাইয়া স্টোর,22.785428,89.974480,3.0,1.0,NaN,"QXPF+5QH, ঘোলারহাট, Bangladesh"


In [ ]:
restaurant_df.describe()

,Latitude,Longitude,Rating,Number of Reviews,Price Level
count,8933.000000,8933.000000,8933.000000,6815.000000,817.000000
mean,23.878697,90.310586,3.094515,95.813940,1.772338
std,1.071477,1.025276,1.852131,456.222281,0.570576
min,20.857776,88.124215,0.000000,1.000000,1.000000
25%,23.130512,89.538739,1.000000,2.000000,1.000000
50%,23.775931,90.378278,3.900000,6.000000,2.000000
75%,24.661437,91.008467,4.300000,34.500000,2.000000
max,26.488103,92.438711,5.000000,23421.000000,4.000000


In [ ]:
len(restaurant_df)

8933